## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/artifacts/0', creation_time=1754231871444, experiment_id='0', last_update_time=1754231871444, lifecycle_stage='active', name='Default', tags={}>]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models") #Log model
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/08/03 14:38:45 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/08/03 14:38:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 14:38:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: '/workspaces/mlops-zoomcamp/artifacts/1/f06462e373ad4e43b1f1d3ec354a8791/artifacts'
🏃 View run defiant-snake-101 at: http://127.0.0.1:5000/#/experiments/1/runs/f06462e373ad4e43b1f1d3ec354a8791
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/artifacts/1', creation_time=1754231925732, experiment_id='1', last_update_time=1754231925732, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/artifacts/0', creation_time=1754231871444, experiment_id='0', last_update_time=1754231871444, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [15]:
from mlflow.tracking import MlflowClient

# Initialize client
client = MlflowClient("http://127.0.0.1:5000")

# First create the registered model (if it doesn't exist)
try:
    client.create_registered_model("linear_reg")
except Exception as e:
    print(f"Model may already exist: {e}")

# Register model version - CORRECTED SOURCE FORMAT
result = client.create_model_version(
    name="linear_reg",
    source="runs:/f06462e373ad4e43b1f1d3ec354a8791/models",  # Note the "runs:/" prefix
    run_id="f06462e373ad4e43b1f1d3ec354a8791"
)

print(f"Model version {result.version} created")


2025/08/03 14:50:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: linear_reg, version 1


Model may already exist: RESOURCE_ALREADY_EXISTS: Registered Model (name=linear_reg) already exists.
Model version 1 created


In [16]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1754232461389, deployment_job_id='', deployment_job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', description='', last_updated_timestamp=1754232611985, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1754232611985, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1754232611985, metrics=None, model_id=None, name='linear_reg', params=None, run_id='f06462e373ad4e43b1f1d3ec354a8791', run_link='', source='runs:/f06462e373ad4e43b1f1d3ec354a8791/models', status='READY', status_message=None, tags={}, user_id='', version='1'>], name='linear_reg', tags={}>]

In [ ]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id # Trouver le run_id automatiquement (dernier run de l'expérience 1)

# Enregistrement du modèle (crée le modèle si inexistant)
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models", # Chemin standard des artefacts
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/08/03 14:57:26 WARNING mlflow.tracking._model_registry.fluent: Run with id f06462e373ad4e43b1f1d3ec354a8791 has no artifacts at artifact path 'models', registering model based on models:/m-7731fd64182e46f1aa2761c58e7e79cb instead
2025/08/03 14:57:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1754233046754, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1754233046754, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='f06462e373ad4e43b1f1d3ec354a8791', run_link='', source='models:/m-7731fd64182e46f1aa2761c58e7e79cb', status='READY', status_message=None, tags={}, user_id='', version='1'>